PWC-Net-small model finetuning (with cyclical learning rate schedule)
=======================================================

In this notebook we:
- Use a small model (no dense or residual connections), 6 level pyramid, uspample level 2 by 4 as the final flow prediction
- Train the PWC-Net-small model on a mix of the `FlyingChairs` and `FlyingThings3DHalfRes` dataset using a Cyclic<sub>short</sub> schedule of our own
- Let the Cyclic<sub>short</sub> schedule oscillate between `2e-05` and `1e-06` for 200,000 steps
- Switch to the "robust" loss described in the paper, instead of the "multiscale" loss used during training

Below, look for `TODO` references and customize this notebook based on your own needs.

## Reference

[2018a]<a name="2018a"></a> Sun et al. 2018. PWC-Net: CNNs for Optical Flow Using Pyramid, Warping, and Cost Volume. [[arXiv]](https://arxiv.org/abs/1709.02371) [[web]](http://research.nvidia.com/publication/2018-02_PWC-Net%3A-CNNs-for) [[PyTorch (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/PyTorch) [[Caffe (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/Caffe)

In [9]:
"""
pwcnet_finetune.ipynb

PWC-Net model finetuning.

Written by Phil Ferriere

Licensed under the MIT License (see LICENSE for details)

Tensorboard:
    [win] tensorboard --logdir=E:\\repos\\tf-optflow\\tfoptflow\\pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned
    [ubu] tensorboard --logdir=/media/EDrive/repos/tf-optflow/tfoptflow/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned
"""
from __future__ import absolute_import, division, print_function
import sys
from copy import deepcopy

from dataset_base import _DEFAULT_DS_TRAIN_OPTIONS#_DEFAULT_DS_TUNE_OPTIONS
from dataset_flyingchairs import FlyingChairsDataset
from dataset_hd1k import HD1KDataset
from dataset_flyingthings3d import FlyingThings3DHalfResDataset
from dataset_mixer import MixedDataset
from model_pwcnet import ModelPWCNet, _DEFAULT_PWCNET_TRAIN_OPTIONS#_DEFAULT_PWCNET_FINETUNE_OPTIONS

## TODO: Set this first!

In [10]:
# TODO: You MUST set dataset_root to the correct path on your machine!
if sys.platform.startswith("win"):
    _DATASET_ROOT = 'E:/datasets/'
else:
    _DATASET_ROOT = '/opt/dataset/'
_HD1K_ROOT = _DATASET_ROOT + 'hd1k_full_package'
#_FLYINGCHAIRS_ROOT = _DATASET_ROOT + 'FlyingChairs_release'
#_FLYINGTHINGS3DHALFRES_ROOT = _DATASET_ROOT + 'FlyingThings3D_HalfRes'
    
# TODO: You MUST adjust the settings below based on the number of GPU(s) used for training
# Set controller device and devices
# A one-gpu setup would be something like controller='/device:GPU:0' and gpu_devices=['/device:GPU:0']
# Here, we use a dual-GPU setup, as shown below
# gpu_devices = ['/device:GPU:0', '/device:GPU:1']
# controller = '/device:CPU:0'
gpu_devices = ['/device:GPU:0']
controller = '/device:GPU:0'

# TODO: You MUST adjust this setting below based on the amount of memory on your GPU(s)
# Batch size
batch_size = 8

# Finetune on `FlyingChairs+FlyingThings3DHalfRes` mix

## Load the dataset

In [11]:
# TODO: You MUST set the batch size based on the capabilities of your GPU(s) 
#  Load train dataset
ds_opts = deepcopy(_DEFAULT_DS_TRAIN_OPTIONS)#_DEFAULT_DS_TUNE_OPTIONS)
ds_opts['in_memory'] = False                          # Too many samples to keep in memory at once, so don't preload them
ds_opts['aug_type'] = 'heavy'                         # Apply all supported augmentations
ds_opts['batch_size'] = batch_size * len(gpu_devices) # Use a multiple of 8; here, 16 for dual-GPU mode (Titan X & 1080 Ti)
ds_opts['crop_preproc'] = (256, 448)                  # Crop to a smaller input size
#ds_opts['type'] = 'into_future'
ds_opts['type'] = 'occ'
ds = HD1KDataset(mode='train_with_val', ds_root=_HD1K_ROOT, options=ds_opts)
#ds2 = FlyingThings3DHalfResDataset(mode='train_with_val', ds_root=_FLYINGTHINGS3DHALFRES_ROOT, options=ds_opts)
#ds = MixedDataset(mode='train_with_val', datasets=[ds1, ds2], options=ds_opts)

In [12]:
# Display dataset configuration
ds.print_config()


Dataset Configuration:
  verbose              False
  in_memory            False
  crop_preproc         (256, 448)
  scale_preproc        None
  tb_test_imgs         False
  random_seed          1969
  val_split            0.03
  aug_type             heavy
  aug_labels           True
  fliplr               0.5
  flipud               0.5
  translate            (0.5, 0.05)
  scale                (0.5, 0.05)
  batch_size           8
  type                 occ
  mode                 train_with_val
  train size           1015
  val size             32


## Configure the finetuning

In [14]:
# Start from the default options
nn_opts = deepcopy(_DEFAULT_PWCNET_TRAIN_OPTIONS)#_DEFAULT_PWCNET_FINETUNE_OPTIONS)
nn_opts['verbose'] = True
#nn_opts['ckpt_path'] = './models/pwcnet-lg-6-2-multisteps-chairsthingsmix_1/pwcnet.ckpt-595000'
nn_opts['ckpt_dir'] = './pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/'
nn_opts['batch_size'] = ds_opts['batch_size']
nn_opts['x_shape'] = [2, ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 3]
nn_opts['y_shape'] = [ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 2]
nn_opts['use_tf_data'] = True # Use tf.data reader
nn_opts['gpu_devices'] = gpu_devices
nn_opts['controller'] = controller

# Use the PWC-Net-small model in quarter-resolution mode
nn_opts['use_dense_cx'] = True
nn_opts['use_res_cx'] = True
nn_opts['pyr_lvls'] = 6
nn_opts['flow_pred_lvl'] = 2

# Robust loss as described doesn't work, so try the following:
#nn_opts['loss_fn'] = 'loss_multiscale' # 'loss_multiscale' # 'loss_robust' # 'loss_robust'
#nn_opts['q'] = 1. # 0.4 # 1. # 0.4 # 1.
#nn_opts['epsilon'] = 0. # 0.01 # 0. # 0.01 # 0.

In [15]:
# Set the learning rate schedule. This schedule is for a single GPU using a batch size of 8.
# Below,we adjust the schedule to the size of the batch and the number of GPUs.
nn_opts['lr_policy'] = 'multisteps'
nn_opts['init_lr'] = 1e-05
nn_opts['lr_boundaries'] = [80000, 120000, 160000, 200000]
nn_opts['lr_values'] = [1e-05, 5e-06, 2.5e-06, 1.25e-06, 6.25e-07]
nn_opts['max_steps'] = 200000

# Below,we adjust the schedule to the size of the batch and our number of GPUs (2).
#nn_opts['max_steps'] = int(nn_opts['max_steps'] * 8 / ds_opts['batch_size'])
#nn_opts['cyclic_lr_stepsize'] = int(nn_opts['cyclic_lr_stepsize'] * 8 / ds_opts['batch_size'])
nn_opts['max_to_keep'] = 50
nn_opts['display_step'] = 1000

In [16]:
# Instantiate the model and display the model configuration
nn = ModelPWCNet(mode='train_with_val', options=nn_opts, dataset=ds)
nn.print_config()

Building model...
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
... model built.
Configuring training ops...
... training ops configured.
Initializing model with random values for initial training...

... model initialized

Model Configuration:
  verbose                True
  ckpt_dir               ./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/
  max_to_keep            50
  x_dtype                <dtype: 'float32'>
  x_shape                [2, 256, 448, 3]
  y_dtype                <dtype: 'float32'>
  y_shape                [256, 448, 2]
  train_mode             train
  adapt_info             None
  sparse_gt_flow         False
  display_step           1000
  snapshot_step          1000
  val_step               1000
  val_batch_size         -1
  tb_val_imgs            pyramid
  tb_test_imgs           None
  gpu_devices            ['/device:GPU:0']
  controller             /device:GPU:0
  use_tf_data            True
  use_mixed_precision    False


## Finetune the model

In [ ]:
# Train the model
nn.train()

Start training from scratch...
2019-02-26 18:26:04 Iter 1000 [Train]: loss=143.65, epe=11.59, lr=0.000010, samples/sec=9.7, sec/step=0.821, eta=1 day, 21:24:11
2019-02-26 18:26:13 Iter 1000 [Val]: loss=98.23, epe=7.88
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-1000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-1000
2019-02-26 18:39:57 Iter 2000 [Train]: loss=145.00, epe=11.73, lr=0.000010, samples/sec=10.1, sec/step=0.791, eta=1 day, 19:28:49
2019-02-26 18:40:00 Iter 2000 [Val]: loss=119.07, epe=9.65
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-2000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-2000
2019-02-26 18:53:45 Iter 3000 [Train]: loss=141.88, epe=11.49, lr=0.000010, samples/sec=10.1, sec/step=

... model saved in ./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-19000
2019-02-26 22:46:56 Iter 20000 [Train]: loss=99.51, epe=7.34, lr=0.000010, samples/sec=10.1, sec/step=0.794, eta=1 day, 15:42:48
2019-02-26 22:46:58 Iter 20000 [Val]: loss=87.87, epe=6.89
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-20000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-20000
2019-02-26 23:00:40 Iter 21000 [Train]: loss=96.23, epe=7.05, lr=0.000010, samples/sec=10.1, sec/step=0.794, eta=1 day, 15:28:26
2019-02-26 23:00:43 Iter 21000 [Val]: loss=69.68, epe=5.15
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-21000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-21000
2019-02-26 23:14:26 Iter 22000 [Train]: lo

... model saved in ./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-38000
2019-02-27 03:07:41 Iter 39000 [Train]: loss=79.41, epe=5.42, lr=0.000010, samples/sec=10.1, sec/step=0.795, eta=1 day, 11:31:58
2019-02-27 03:07:44 Iter 39000 [Val]: loss=69.12, epe=5.00
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-39000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-39000
2019-02-27 03:21:26 Iter 40000 [Train]: loss=76.56, epe=5.20, lr=0.000010, samples/sec=10.1, sec/step=0.795, eta=1 day, 11:19:43
2019-02-27 03:21:29 Iter 40000 [Val]: loss=71.55, epe=5.21
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-40000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-40000
2019-02-27 03:35:11 Iter 41000 [Train]: lo

... model saved in ./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-57000
2019-02-27 07:27:24 Iter 58000 [Train]: loss=69.54, epe=4.54, lr=0.000010, samples/sec=10.2, sec/step=0.787, eta=1 day, 7:02:24
2019-02-27 07:27:26 Iter 58000 [Val]: loss=84.89, epe=6.16
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-58000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-58000
2019-02-27 07:41:02 Iter 59000 [Train]: loss=68.85, epe=4.49, lr=0.000010, samples/sec=10.1, sec/step=0.788, eta=1 day, 6:52:14
2019-02-27 07:41:04 Iter 59000 [Val]: loss=82.68, epe=6.54
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-59000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-multisteps-chairsthingsmix_1_train/pwcnet.ckpt-59000
2019-02-27 07:54:39 Iter 60000 [Train]: loss

## Training log

Here are the training curves for the run above:

![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/loss.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/epe.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/lr.png)

Here are the predictions issued by the model for a few validation samples:

![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val1.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val2.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val3.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val4.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val5.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val6.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val7.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val8.png)